<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/Data_collection_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import moduls

In [ ]:
!pip install wikidata --quiet
!pip install datasets --quiet


import pandas as pd
import requests
from wikidata.client import Client
import time
from tqdm import tqdm

# Import of the training data
from google.colab import files
uploaded = files.upload()

df_train = pd.read_csv('/content/[MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv', sep = '\t')

# validation data

from datasets import load_dataset
from huggingface_hub import login

login(token="")

dataset = load_dataset('sapienzanlp/nlp2025_hw1_cultural_dataset')
df_dev = dataset['validation'].to_pandas()


# Function to extract the text

Functions to extract the Wikipedia text for a given item

In [ ]:
def extract_entity_id(url):
    return url.strip().split("/")[-1]

def get_wikipedia_extract(wikidata_url):
    """return the Wikipedia text for a given wikidata url"""

    entity_id = extract_entity_id(wikidata_url)

    client = Client()
    try:
        item = client.get(entity_id, load=True)
        # Get english wikipedia sitelink
        sitelinks = item.data.get("sitelinks", {})
        enwiki = sitelinks.get("enwiki")

        if enwiki:
            title = enwiki["title"]

            #Wikipedia API to get the text
            api_url = "https://en.wikipedia.org/w/api.php"
            headers = {'User-Agent': 'Mozilla/5.0'}
            params = {
                "action": "query",
                "prop": "extracts",
                "explaintext": True,
                "titles": title,
                "format": "json",
                "redirects": 1
            }

            res = requests.get(api_url, params=params, headers=headers).json()
            page = next(iter(res["query"]["pages"].values()))
            return page.get("extract", "No extract found.")[:1000] #1000 caracters
        else:
            return "No English Wikipedia article found for this entity."
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            print(f"Entity not found for URL: {wikidata_url}")
            return "Entity not found in Wikidata."
        else:
            raise e  # Re-raise other HTTP errors
    except Exception as e:
        # Catch any other exception that may have occurred during Wikidata entity retrieval
        print(f"An unexpected error occurred for URL: {wikidata_url}. Error: {e}")
        return "Error retrieving Wikidata entity."

# Application to the train dataset

In [ ]:
# Application to the dataset
tqdm.pandas()
df_train['text'] = df_train['item'].progress_apply(get_wikipedia_extract)

 33%|███▎      | 2061/6251 [05:19<08:49,  7.91it/s]

An unexpected error occurred for URL: http://www.wikidata.org/entity/Q7551241. Error: HTTP Error 404: Not Found


100%|██████████| 6251/6251 [16:39<00:00,  6.25it/s]


In [ ]:
# Checking item without text
df_train[(df_train['text'] == 'Error retrieving Wikidata entity.') |
          (df_train['text'] == "No English Wikipedia article found for this entity.")]

,item,name,description,type,category,subcategory,label,text
2058,http://www.wikidata.org/entity/Q7551241,social media and television,Emerging platforms,entity,media,television,cultural agnostic,Error retrieving Wikidata entity.
2718,http://www.wikidata.org/entity/Q6394187,Kernel Normal Form,Coding style used in the development of code f...,entity,literature,writing style,cultural exclusive,No English Wikipedia article found for this en...


#Application to the dev set

In [ ]:
# Application to the dev set
df_dev['text'] = df_dev['item'].progress_apply(get_wikipedia_extract)

100%|██████████| 300/300 [00:51<00:00,  5.88it/s]


In [ ]:
# Checking item without text
df_dev[(df_dev['text'] == 'Error retrieving Wikidata entity.') |
          (df_dev['text'] == "No English Wikipedia article found for this entity.")]

,item,name,description,type,category,subcategory,label,text


# Exportation

In [ ]:
df_train.to_json('train_with_text.json', orient='records', lines=True)
df_dev.to_json('dev_with_text.json', orient='records', lines=True)

from google.colab import files

files.download('train_with_text.json')
files.download('dev_with_text.json')